In [ ]:
# Reduce

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("reduction-op")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/29 11:14:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/29 11:15:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/29 11:15:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/08/29 11:15:02 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/08/29 11:15:02 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


# Reduce
- 사용자가 지정하는 함수를 받아(task) 여러 개의 값을 하나로 줄여준다.
- 파티션 별로 작업이 일어난다.

In [2]:
from operator import add

In [3]:
add(1,2)

3

In [4]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [5]:
sample_rdd.reduce(add)

15

In [6]:
sample_rdd_p1 = sc.parallelize([1, 2, 3, 4])

In [7]:
sample_rdd_p1.glom().collect()

[[1, 2, 3, 4]]

In [8]:
sample_rdd_p2 = sc.parallelize([1, 2, 3, 4], 2)

In [9]:
sample_rdd_p2.glom().collect()

[[1, 2], [3, 4]]

In [10]:
# 파티션이 1개인 상태에서 reduce
sample_rdd_p1.reduce(lambda x, y : (x * 2) + y)

26

In [11]:
# [[1, 2, 3, 4]]
# 1*2+2=4
# 4*2+3=11
# 11*2+4=26

In [12]:
sample_rdd_p2.reduce(lambda x, y : (x * 2) + y)

18

In [13]:
# [[1, 2], [3, 4]]
# 1*2 + 2 = 4
# 3*2 + 4 = 10
# 4*2 + 10 = 18

In [14]:
sample_rdd_p3 = sc.parallelize([1, 2, 3, 4], 3) # 파티션이 3개
sample_rdd_p3.reduce(lambda x, y : (x * 2) + y)

18

In [15]:
sample_rdd_p3.glom().collect()

[[1], [2], [3, 4]]

In [16]:
sample_rdd_p4 = sc.parallelize([1, 2, 3, 4], 4) # 파티션이 4개
sample_rdd_p4.reduce(lambda x, y : (x * 2) + y)

26

In [17]:
sample_rdd_p4.glom().collect()

[[1], [2], [3], [4]]

In [18]:
# [[1], [2], [3], [4]]
# [1,2,3,4]

In [19]:
# [[1], [2], [3, 4]]
# Partition 1: 1 

# Partition 2: 2 

# Partition 3: 3*2 + 4 = 10

# [1,2,10]
# 1*2 + 2 = 4
# 2*2 + 10 = 14

reduce 연산의 함수가 **비결합 법칙(Non-associative law)**을 따를 경우, 파티션의 개수와 데이터 분배 방식에 따라 결과값이 달라질 수 있습니다. 
하지만 Spark는 대규모 데이터 처리를 위해 병렬화를 극대화하는 것을 목표로 하며, 

- 파티션을 사용하는 주요 이유는 다음과 같습니다.

1. 병렬 처리 (Parallelism)
파티션은 데이터를 논리적으로 분할하여 여러 코어 또는 머신에서 동시에 작업을 수행할 수 있게 합니다. 
RDD의 각 파티션은 개별 태스크로 처리되므로, 여러 태스크가 동시에 실행되어 전체 연산 시간을 획기적으로 단축시킬 수 있습니다. 
    이것이 분산 컴퓨팅의 핵심 이점입니다.

2. 데이터 지역성 (Data Locality)
파티션을 사용하면 데이터를 연산이 수행되는 노드에 가깝게 배치할 수 있습니다. 
예를 들어, HDFS에 저장된 데이터는 여러 블록으로 나뉘어 클러스터의 각 노드에 분산됩니다. 
Spark는 이 블록의 위치 정보를 파티션에 매핑하여, 데이터를 네트워크를 통해 전송하는 대신 데이터가 있는 노드에서 직접 연산을 수행하게 합니다. 
이를 통해 I/O 비용을 크게 줄일 수 있습니다.

3. 결함 허용 (Fault Tolerance)
파티션은 Spark의 결함 허용(Fault Tolerance) 시스템의 기본 단위입니다. 
만약 클러스터의 한 노드에서 실패가 발생하더라도, 해당 노드의 파티션만 재계산하면 되기 때문에 전체 작업을 처음부터 다시 시작할 필요가 없습니다.

결론: 파티션과 reduce 함수의 관계
reduce 연산에서 파티션의 개수에 따라 결과가 달라지는 문제는 lambda x, y: (x * 2) + y와 같이 결합 법칙이 성립하지 않는 연산자를 사용했기 때문에 발생합니다. 
Spark는 이러한 함수를 사용할 때 개발자에게 주의를 요하지만, Spark의 근본적인 목적은 병렬 처리를 통한 성능 향상에 있습니다. 
따라서 대부분의 reduce 연산에는 결합 법칙이 성립하는 연산(덧셈, 곱셈 등)을 사용하도록 권장하여 병렬 처리의 이점을 온전히 누리게 합니다.

결과 값이 다르다면 파티션의 정확한 계수를 어떻게 산정하는가

reduce 연산처럼 결합 법칙이 성립하지 않는 함수를 사용할 경우, 파티션 수에 따라 결과가 달라지므로 파티션 수를 1개로 설정하여 순차적으로 계산하는 것이 정확한 결과를 보장하는 유일한 방법입니다.

# Fold
- fold()는 단순 reduce와 달리 각 파티션에서 먼저 "초기값"을 적용한 후, 최종적으로 또 한 번 초기값을 적용해서 결과를 합칩니다.
- `reduce`와 비슷하지만, `zeroValue`에 넣어 놓고 싶은 시작값을 지정해서 `reduce`
- `zeroValue`는 파티션 마다 계산이 일어날 때 하나씩 더해지는 값

#### fold(zeroValue, func) 동작 방식
- 각 파티션에 대해 zeroValue를 시작값으로 하여 func를 적용해 부분 결과를 구함
- 파티션별 결과들을 다시 zeroValue로 시작해서 func를 적용해 최종 결과를 만듦
- 즉, zeroValue는 단순히 최종 결과에만 한 번 쓰이는 게 아니라 파티션마다 + 마지막에 한 번 더 들어가게 됩니다.

In [26]:
sample_rdd = sc.parallelize([2, 3, 4], 4)
sample_rdd.glom().collect()

[[], [2], [3], [4]]

In [28]:
sample_rdd.reduce(lambda x, y : (x * y))

24

In [29]:
sample_rdd.fold(1, lambda x, y : (x * y))

24

#### 1) fold(1, lambda x, y: x * y)
##### [[], [2], [3], [4]]
- Partition 0: 시작값=1, 데이터 없음 → 결과=1
- Partition 1: 시작값=1 → (1*2)=2
- Partition 2: 시작값=1 → (1*3)=3
- Partition 3: 시작값=1 → (1*4)=4
- → 파티션 결과 = [1, 2, 3, 4]

#### 이제 다시 fold 값 = 1:
- (((1 * 1) * 2) * 3) * 4 = 24

In [30]:
sample_rdd.fold(2, lambda x, y : (x * y))

768

#### 2) fold(2, lambda x, y: x * y)
##### [[], [2], [3], [4]]

- Partition 0: 시작값=2, 데이터 없음 → 결과=2
- Partition 1: 시작값=2 → (2*2)=4
- Partition 2: 시작값=2 → (2*3)=6
- Partition 3: 시작값=2 → (2*4)=8
- → 파티션 결과 = [2, 4, 6, 8]

##### 이제 다시 fold (초기값=2 사용):
(((2 * 2) * 4) * 6) * 8
= (4 * 4) * 6 * 8
= 16 * 6 * 8
= 96 * 8
= 768

#### 결론
- fold는 파티션마다 zeroValue가 주입되므로, reduce와는 다르게 결과가 크게 달라질 수 있음.
- zeroValue가 곱셈에 대한 항등원(1)일 때는 reduce랑 같은 결과가 나오지만, 2처럼 항등원이 아닌 값을 넣으면 결과가 달라집니다.

# groupBy
- 그룹핑할 기준 함수를 받아서 reduction

#### 1. groupBy 동작 원리
- groupBy(func) 는 각 원소에 대해 func 결과를 키(key)로 묶어줍니다.
- 여기서는 lambda x: x % 2 이므로 짝수/홀수 여부(0, 1) 를 기준으로 그룹화가 됩니다.
- 반환 결과는 RDD of (key, iterable) 형태입니다.

#### 2. 데이터 분류
- 원본 데이터: [1, 1, 2, 3, 5, 6, 7, 10, 3]
- 홀수 (x % 2 = 1) → [1, 1, 3, 5, 7, 3]
- 짝수 (x % 2 = 0) → [2, 6, 10]

In [31]:
sample_rdd = sc.parallelize([1, 1, 2, 3, 5, 6, 7, 10, 3])

In [32]:
result = sample_rdd.groupBy(lambda x : x % 2).collect()
result

[(1, <pyspark.resultiterable.ResultIterable at 0x723ebff33580>),
 (0, <pyspark.resultiterable.ResultIterable at 0x723ebff33850>)]

In [33]:
# sorted : 내부 Iterable 객체를 정렬한 후 리스트로 리턴
# 키 정렬 + 그룹 내부 정렬까지 적용된 리스트
sorted([(x, sorted(y)) for (x, y) in result]) 

[(0, [2, 6, 10]), (1, [1, 1, 3, 3, 5, 7])]

# Aggregate
- `RDD.aggregate(zeroValue, seqOp, combOp)`
    - `zeroValue` : 각 파티션에서 누적할 시작 값
    - `seqOp` : 타입 변경 함수
        - 파티션 내에서 벌어지는 연산을 담당
    - `combOp` : 모든 결과를 하나로 합쳐주는 역할
- 파티션 단위의 연산 결과를 합쳐주는 과정을 거치게 된다.

In [47]:
sample_rdd = sc.parallelize([1, 2, 3, 4], 2)
sample_rdd

ParallelCollectionRDD[34] at readRDDFromFile at PythonRDD.scala:287

1️⃣ 파티션 확인

In [48]:
sample_rdd.glom().collect()

[[1, 2], [3, 4]]

- 파티션 1: [1, 2]
- 파티션 2: [3, 4]

In [42]:
seqOp = lambda x, y : (x[0] + y, x[1] + 1) # 파티션 내의 연산
# seqOp

### 2️⃣ 각 파티션에 seqOp 적용
##### 초기값: (0, 0)
- 파티션 1: [1, 2]
  - (0,0) + 1 → (0+1, 0+1) = (1,1)
  - (1,1) + 2 → (1+2, 1+1) = (3,2)
  - 파티션 1 결과: (3,2)

- 파티션 2: [3, 4]
  -  (0,0) + 3 → (0+3,0+1) = (3,1)
  - (3,1) + 4 → (3+4,1+1) = (7,2)
  - 파티션 2 결과: (7,2)

### 3️⃣ combOp로 파티션 결과 합치기

In [40]:
combOp = lambda x, y : (x[0] + y[0], x[1] + y[1]) # 파티션의 모든 결과를 최종 연산

- (3,2) + (7,2) → (3+7, 2+2) = (10,4)

✅ 최종 결과

In [44]:
sample_rdd.aggregate((0, 0), seqOp, combOp)

(10, 4)

- 합계: 10
- 개수: 4